## Tekstide indekseerimine


[![License: BSD-2-Clause](https://img.shields.io/badge/License-BSD--2--Clause-lightgrey.svg)](https://opensource.org/license/bsd-2-clause/)


Järgnevas kasutame tekstide indekseerimiseks spetsiifilist veebiteenust, mis väljastab peale tekstides sisalduvate sõnade algvormide palju muud morfoloogilist informatsiooni, mida läheb meil edaspidi päringulaiendaja loomiseks vaja.


Dokumentide indekseerimiseks võib analüüsida kas originaaltekste või neile vastavaid puhastatud tekste. 
Puhastatud tekstide analüüsimisel väljundindeksis olevad sõnavormide asukohad ei pruugi vastata sõnade tegelikule asukohale originaaltekstis, sest teksti täiendamine lisab teksti sümboleid. 

Kuna meie lõppeesmärk indeksi loomisel on vaid otsitavate sõnavormide leidmine mitte nende täpse asukoha talletamine, siis me ignoreerime antud probleemi ja kasutame indekseerimiseks puhastatud tekste.


In [1]:
import json
import math
import requests

from pandas import DataFrame 
from pandas import read_csv
from tqdm.auto import tqdm
from typing import List

## I. Indekseerimisteenuse dokumenthaaval kasutamine

Veebiteenuse kasutamine üks dokument korraga on sobilik juhul, kui dokumendid on pikad või on neid piisavalt palju, et ühe liitpäringu tegemine on liiga koormav. Näiteks siis, kui indekseeritakse seaduse tekste. 
Kuna järgnevad sammud eeldavad, et väljundindeksis olev `DOCID` väli vastab dokumendi globaalsele indeksile, siis päringu tegemisel tuleb määrata nii tekst kui ka sellele vastav `global_id` väli Riigi Teataja infosüsteemis.
Vaikimisi annab teenus tagasi täpselt kolm tabelit, mida on päringulaiendaja loomiseks vaja. 
Vajadusel saab teisi tabeleid juurde tellida.


In [2]:
def index_document(doc_id: str, text: str, output_tables: List[str] = None):
    """
    Uses web service to index document for further processing.

    Returns a JSON object that can be further modified or serialised to text.
    All indices are inside the field 'tabelid' which contains up to five subfields.
    The argument output_tables specifies which of them are present in the output.  

    Two out of these correspond to actual word locations:
    - Subfield 'indeks_lemmad' contains information about lemmas in an unspecified order. 
    - Subfield 'indeks_vormid' contains information about wordforms in an unspecified order. 

    The remaining three contain aggregated information about the document:  
    - Subfield 'liitsõnad' contains what subwords compound words in the document contain. 
    - Subfield 'lemma_kõik_vormid' contains all potential wordform for each lemma.
    - Subfield 'lemma_korpuse_vormid' contains all wordform for each lemma that exists in the document.
    """
    if output_tables is None:
        output_tables = ['lemma_kõik_vormid', 'lemma_korpuse_vormid', 'liitsõnad']
        
    ANALYZER_QUERY =  "https://smart-search.tartunlp.ai/api/advanced_indexing/json" 
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {"params":{"tables": output_tables}, "sources": {str(doc_id): {"content": text}}}
    
    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    return response.json()

In [3]:
text = \
"""
Kõikumatus usus ja vankumatus tahtes kindlustada ja arendada riiki, 
mis on loodud Eesti rahva riikliku enesemääramise kustumatul õigusel ja välja kuulutatud 1918. aasta 24. veebruaril, 
mis on rajatud vabadusele, õiglusele ja õigusele, 
mis on kaitseks sisemisele ja välisele rahule ning pandiks praegustele ja tulevastele põlvedele nende ühiskondlikus edus ja üldises kasus, 
mis peab tagama eesti rahvuse, keele ja kultuuri säilimise läbi aegade – võttis Eesti rahvas 1938. aastal jõustunud põhiseaduse paragrahv 1 
alusel 1992. aasta 28. juuni rahvahääletusel vastu järgmise põhiseaduse.
"""
tabelid = ['lemma_kõik_vormid', 'lemma_korpuse_vormid', 'liitsõnad', 'indeks_lemmad', 'indeks_vormid', 'kirjavead']
result = index_document(115052015002, text, tabelid)

other_result = index_document(115052015002, 'Punameremao pisarad.', tabelid)

### Lemmade indeksi kirjeldus näidetena

Kuna ühel sõnal võib olla mitu võimalikku algvormi, siis ühele lokatsioonile võib vastata mitu lemmat.
Iga tabeli rea korral näitab kaal selle tõenäosust indekseerimisteenuse arvates. 
Vaikimisi saavad kõik sõnavormile vastavad algvormid sama kaalu ning kaalude summa on üks.  

In [4]:
lemma_index = DataFrame(result['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'weight', 'is_sublemma'])
display(lemma_index.sort_values('start').head(5))
display(lemma_index.sort_values('start').head(5))

,lemma,doc_id,start,end,weight,is_sublemma
1,kõikumatu,115052015002,1,11,0.5,False
79,kõikumatus,115052015002,1,11,0.5,False
19,usk,115052015002,12,16,1.0,False
94,vankumatu,115052015002,20,30,0.5,False
5,vankumatus,115052015002,20,30,0.5,False


,lemma,doc_id,start,end,weight,is_sublemma
1,kõikumatu,115052015002,1,11,0.5,False
79,kõikumatus,115052015002,1,11,0.5,False
19,usk,115052015002,12,16,1.0,False
94,vankumatu,115052015002,20,30,0.5,False
5,vankumatus,115052015002,20,30,0.5,False


In [5]:
idx1 = lemma_index[lemma_index['lemma']=='põhiseadus'].reset_index().iloc[0,0]
print(f"Lemma:           {lemma_index.loc[idx1, 'lemma']}")
print(f"Lokatsioon:      [{lemma_index.loc[idx1, 'start']}, {lemma_index.loc[idx1, 'end']})")
print(f"Vastav sõnavorm: {text[lemma_index.loc[idx1, 'start']:lemma_index.loc[idx1, 'end']]}")

Lemma:           põhiseadus
Lokatsioon:      [580, 591)
Vastav sõnavorm: põhiseaduse


In [6]:
lemma_index[lemma_index['is_sublemma']].sort_values('start').style.set_caption("Liitsõna alamosad")

,lemma,doc_id,start,end,weight,is_sublemma
78,määramine,115052015002,105,119,1.000000,True
80,ise,115052015002,105,119,1.000000,True
34,põhi,115052015002,495,506,1.000000,True
64,seadus,115052015002,495,506,1.000000,True
48,hääletus,115052015002,549,564,1.000000,True
57,rahvas,115052015002,549,564,1.000000,True
74,põhi,115052015002,580,591,1.000000,True
91,seadus,115052015002,580,591,1.000000,True


In [7]:
idx2 = lemma_index[(lemma_index['lemma']=='põhi') & (lemma_index['start'] == lemma_index.loc[idx1, 'start'])].reset_index().iloc[0,0]
print(f"Lemma:           {lemma_index.loc[idx2, 'lemma']}")
print(f"Lokatsioon:      [{lemma_index.loc[idx2, 'start']}, {lemma_index.loc[idx2, 'end']})")
print(f"Vastav sõnavorm: {text[lemma_index.loc[idx2, 'start']: lemma_index.loc[idx2, 'end']]}")

Lemma:           põhi
Lokatsioon:      [580, 591)
Vastav sõnavorm: põhiseaduse


**Oluline:** Kuna erienevatel alamsõnadel võib olla erinev arv algvorme võivad kaalud olla erinevad. 

In [8]:
tbl = DataFrame(other_result['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'weight', 'is_sublemma'])
tbl[tbl['is_sublemma']].sort_values(['start', 'lemma']).style.set_caption("Liitsõna alamosad")

,lemma,doc_id,start,end,weight,is_sublemma
7,madu,115052015002,0,11,0.500000,True
1,magu,115052015002,0,11,0.500000,True
3,meremadu,115052015002,0,11,0.500000,True
5,meremagu,115052015002,0,11,0.500000,True
8,meri,115052015002,0,11,1.000000,True
2,puna,115052015002,0,11,0.500000,True
0,punama,115052015002,0,11,0.500000,True
4,punameri,115052015002,0,11,1.000000,True


### Sõnavormide indeksi kirjeldus näidetena

Sõnavormide puhul võimalikku mitmesust ei ole. Ühele lokatsioonile vastab ainult üks sõnavorm.

In [9]:
wordform_index = DataFrame(result['tabelid']['indeks_vormid'], columns=['wordform', 'doc_id', 'start', 'end', 'is_sublemma'])
display(wordform_index.head(5))
display(wordform_index.sort_values('start').head(5))

,wordform,doc_id,start,end,is_sublemma
0,kaitseks,115052015002,246,254,False
1,usus,115052015002,12,16,False
2,24.,115052015002,171,174,False
3,põhi,115052015002,580,591,True
4,jõustunud,115052015002,485,494,False


,wordform,doc_id,start,end,is_sublemma
36,kõikumatus,115052015002,1,11,False
1,usus,115052015002,12,16,False
35,vankumatus,115052015002,20,30,False
37,tahtes,115052015002,31,37,False
74,kindlustada,115052015002,38,49,False


In [10]:
print(f"Lemma:           {wordform_index.loc[1, 'wordform']}")
print(f"Lokatsioon:      [{wordform_index.loc[1, 'start']}, {wordform_index.loc[1, 'end']})")
print(f"Vastav sõnavorm: {text[wordform_index.loc[1, 'start']:wordform_index.loc[1, 'end']]}")


Lemma:           usus
Lokatsioon:      [12, 16)
Vastav sõnavorm: usus


In [11]:
start = wordform_index.loc[wordform_index['wordform']=='rahvahääletusel', 'start'].iloc[0]
display(wordform_index[wordform_index['start'] == start].sort_values('wordform', ascending=True).style.set_caption("Rahvahääletuse alamsõnavormid"))
display(lemma_index[lemma_index['start'] == start].sort_values('is_sublemma', ascending=True).style.set_caption("Rahvahääletuse alamsõnavormide lemmad"))

,wordform,doc_id,start,end,is_sublemma
69,hääletusel,115052015002,549,564,True
24,rahva,115052015002,549,564,True
66,rahvahääletusel,115052015002,549,564,False


,lemma,doc_id,start,end,weight,is_sublemma
24,rahvahääletus,115052015002,549,564,1.000000,False
48,hääletus,115052015002,549,564,1.000000,True
57,rahvas,115052015002,549,564,1.000000,True


### Liitsõnade tabeli kirjeldus näidetena 

Kuna ühel sõnal võib olla mitu võimalikku algvormi, siis liitsõna eri osadele võib vastata mitu lemmat.

In [12]:
compound_words = DataFrame(result['tabelid']['liitsõnad'], columns=['sublemma', 'lemma'])
display(compound_words.sort_values('lemma'))

,sublemma,lemma
3,määramine,enesemääramine
4,ise,enesemääramine
2,seaduma,põhiseadus
5,seadus,põhiseadus
6,põhi,põhiseadus
0,hääletu,rahvahääletus
1,rahvas,rahvahääletus
7,hääletus,rahvahääletus


### Lemmade kõikide vormide tabeli kirjeldus näidetena

Selles tabelis on kõikvõimalikud tekstides esinevate sõnade algvormide käänamisel ja pööramisel saadud vormid. Tabelis olev kaal näitab sõnavormi sagedust tekstides.

In [13]:
all_wordforms = DataFrame(result['tabelid']['lemma_kõik_vormid'], columns=['wordform', 'weight', 'lemma'])
display(all_wordforms.sort_values('weight', ascending=False).head(5))

,wordform,weight,lemma
140,Eesti,3,eesti
2365,eesti,3,eesti
2743,olema,3,on
2194,põhiseadus,2,põhiseaduse
1957,Eesti,2,Eesti


### Lemmade kõikide tekstis esinevate vormide tabeli kirjeldus näidetena

Selles tabelis on kõikvõimalikud tekstides esinevate sõnavormide sagedus.


In [14]:
existing_wordforms = DataFrame(result['tabelid']['lemma_korpuse_vormid'], columns=['wordform', 'weight', 'lemma'])
display(existing_wordforms.sort_values('weight', ascending=False).head(5))

,wordform,weight,lemma
63,eesti,3,eesti
65,Eesti,3,eesti
69,olema,3,on
22,põhi,2,põhi
10,aasta,2,aasta


## II. Indekseerimisteenuse kasutamine mitme dokumendi jaoks

Sama veebiteenust saab kasutada ka mitme dokumandi korraga indekseerimiseks. Seda on mõistlik teha vähendamaks veebipäringute arvu ja väljundite summaarse andmemahu vähendamiseks. Ainsaks probleemiks on sisendi suurus. Vaikimisi on veebiteenus seadistatud nii, et kogusisendi suurus ei tohiks olla üle 10 megabaidi. 

**Sisendi suuruse muutmine:** Veebiteenuse kogusisendi piirangu seadistamine käib läbi keskkonnamuutujate. 
Vaata täpsemalt juhendeid:

* [`smart-search/api/api_advanced_indexing/`](../../../../api/api_advanced_indexing/)
* [`smart-search/kube`](../../../../kube/)



In [15]:
def index_documents(doc_ids: List[str], texts: List[str],  output_tables: List[str] = None, size_limit:int = 10 * 10**6):
    """
    Uses web service to index documents for further processing.
    Raises value error if the input creates too long input for the webservice.
    Size limit is given in bytes.

    Returns a JSON object that can be further modified or serialised to text.
    All indices are inside the field 'tabelid' which contains up to five subfields.
    The argument output_tables specifies which of them are present in the output.  

    Two out of these correspond to actual word locations:
    - Subfield 'indeks_lemmad' contains information about lemmas in an unspecified order. 
    - Subfield 'indeks_vormid' contains information about wordforms in an unspecified order. 

    The remaining three contain aggregated information about the document:  
    - Subfield 'liitsõnad' contains what subwords compound words in the document contain. 
    - Subfield 'lemma_kõik_vormid' contains all potential wordform for each lemma.
    - Subfield 'lemma_korpuse_vormid' contains all wordform for each lemma that exists in the document.
    """
    if output_tables is None:
        output_tables = ['lemma_kõik_vormid', 'lemma_korpuse_vormid', 'liitsõnad']

    ANALYZER_QUERY =  "https://smart-search.tartunlp.ai/api/advanced_indexing/json" 
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {
        "params":{"tables": output_tables}, 
        'sources': {id: {'content': text} for id, text in zip(doc_ids, texts)}} 

    # Abort if the input is too long
    byte_size =len(json.dumps(POST_DATA_TEMPLATE, ensure_ascii=False).encode("utf-8"))
    if byte_size > size_limit:
        raise ValueError('Input is too long')

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"

    return response.json()

In [16]:
doc_ids=['25055', '964508', '202122020001', '123042022001']
texts = ["Rahvusvahelise patendiklassifikatsiooni Strasbourg'i kokkuleppega ühinemise seadus", 
         "Tuumaohutuse konventsiooniga ühinemise seadus",
         "Eesti Vabariigi ja Euroopa Tuumauuringute Organisatsiooni(CERN) vahelise CERN-iga "
         "ühinemise eelses staadiumis assotsieerunud liikme staatuse andmist käsitleva kokkuleppe ratifitseerimise seadus",
         "Seadus «Riigi 2002. aasta lisaeelarve»"] 

result = index_documents(doc_ids, texts, output_tables=['indeks_vormid', 'indeks_lemmad'])

In [17]:
wordform_index = DataFrame(result['tabelid']['indeks_vormid'], columns=['wordform', 'doc_id', 'start', 'end', 'is_sublemma'])
display(wordform_index.sort_values('wordform')[:10])

,wordform,doc_id,start,end,is_sublemma
31,2002.,123042022001,14,19,False
32,CERN,202122020001,58,62,False
48,CERNiga,202122020001,73,81,False
50,Eesti,202122020001,0,5,False
51,Euroopa,202122020001,19,26,False
21,Strasbourg'i,25055,40,52,False
43,aasta,123042022001,20,25,False
41,andmist,202122020001,141,148,False
15,arve,123042022001,26,37,True
28,assotsieerunud,202122020001,110,124,False


In [18]:
lemma_index = DataFrame(result['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'weight', 'is_sublemma'])
display(lemma_index.sort_values('lemma')[:10])

,lemma,doc_id,start,end,weight,is_sublemma
24,2002.,123042022001,14,19,1.0,False
61,CERN,202122020001,58,62,1.0,False
3,CERN,202122020001,73,81,1.0,False
15,Eesti,202122020001,0,5,0.5,False
69,Euroopa,202122020001,19,26,0.5,False
59,Strasbourg,25055,40,52,1.0,False
47,aasta,123042022001,20,25,1.0,False
38,andmine,202122020001,141,148,1.0,False
44,arv,123042022001,26,37,0.5,True
30,arve,123042022001,26,37,0.5,True


**Näide pikkusepiirangu rakendumisest**

In [19]:
try:
    index_documents(doc_ids, texts, size_limit=10)
except ValueError as e:
    print(e)

Input is too long


## III. Riigiteataja pealkirjade indekseerimine

Lihtsuse ja selguse mõttes indekseerime dokumente järjestikku kombineerides dokumentide pealkirjad 1000 elemendilisteks gruppideks ning salvestame tulemused kataloogi `results/document_indeks`. Praktikas oleks suuremate mahtude korral mõistlik teha indekseerimist paraleelselt nii, et igale skriptile vastaks oma veebiteenuse lõim. Näiteid selle kohta leiab kataloogist [smart-search/scripts/query_extender_setup/example_script_based_workflow](../../../../scripts/query_extender_setup/example_script_based_workflow/).


In [20]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts/government_orders.csv', header=0)

In [21]:
BATCH_SIZE = 2
OUTPUT_DIR = '../results/document_index/'
for name, source in sources.items():
    print(f'Sisendfail: {name}.csv ({len(source)} rida)')
    
    batch_count = math.ceil(len(source)/BATCH_SIZE)
    for k in tqdm(range(batch_count)):
        batch = source.loc[k*BATCH_SIZE:(k+1)*BATCH_SIZE, ['global_id', 'document_title']]
        result = index_documents(batch['global_id'].astype(str), batch['document_title'])
        with open(f'{OUTPUT_DIR}{name}_{k:03d}.json', 'w') as output:
            json.dump(result, output)    
        break

Sisendfail: state_laws.csv (4730 rida)


  0%|          | 0/2365 [00:00<?, ?it/s]

Sisendfail: government_regulations.csv (12609 rida)


  0%|          | 0/6305 [00:00<?, ?it/s]

Sisendfail: local_government_acts.csv (36525 rida)


  0%|          | 0/18263 [00:00<?, ?it/s]

Sisendfail: government_orders.csv (4973 rida)


  0%|          | 0/2487 [00:00<?, ?it/s]

### Uute pealkirjade indekseerimine

Uute dokumentide lisandumisel tekib vajadus neid indekseerida. Seda saab teha analoogselt pealkirjade indekseerimisega ning seejärel lisada uued andmed päringulaiendajasse. Seda saab põhimõtteliselt teha kahel moel. Esiteks võib teha kogu analüüsi uuest. Teiseks võib teha analüüsi vaid uute dokumentide peal. Indekseerimisväljund on selline, et seda saab kasutada otse olemasoleva päringulaiendaja sisendi uuendamiseks. 

